# Invoke-WebRequest

#### Description 
The 298 Domestic Vs Non-Domestic report is is a month-end claims review report provided for Phoebe Putnam Health System - master group 298. The report offers a claims review snapshot subdivided by domestic claims, bill type, locations, and plans. The EOM report is currently generated manually via WLT claims exports and MS Access. 

The aim is to transfer the logic per Phoebe MS Access database to SQL script to automate the report.

* The MS Access database can be found at: [S:\Depts\Reporting & Analytics\Access Databases\Phoebe.accdb](file:///S:/Depts/Reporting%20&%20Analytics/Access%20Databases/Phoebe.accdb). 
* The report used as a control for data alignment can be found at: [S:\Groups\active\Phoebe Putney Health System (298)\Month End Reports\2020 Month End Reports\January\298 Domestic vs Non Domestic Jan 2019.xlsx](file:///S:/Groups/active/Phoebe%20Putney%20Health%20System%20(298)/Month%20End%20Reports/2020%20Month%20End%20Reports/January/298%20Domestic%20vs%20Non%20Domestic%20Jan%202019.xlsx).
* The reporting period used for comparison is August 1st, 2019 through January 31st 2020. 

> Original request / details: [JIRA PS-2825](https://jira.maestrohealth.com/browse/PS-2825). 

## Step 1 - Get and inspect the data

The Phoebe Database aggregates WLT claims data exports using 4 main tables:

#### 298 Domestic TIN
Provider TIN to categorize domestic claims. 

<img src="https://github.com/fdk1034e/Access-Database-Queries/blob/master/298/298_Domestic_TIN.png?raw=true" width="38%">

#### 298 Locations
Location Name2 to bin sub-group location numbers / active & cobra location names.

<img src="https://github.com/fdk1034e/Access-Database-Queries/blob/master/298/298_Locations.png?raw=true" width="50%">

#### 298 Sub-groups
Plan Name to bin sub-group numbers / names.

<img src="https://github.com/fdk1034e/Access-Database-Queries/blob/master/298/298_Sub_groups.png?raw=true" width="50%">

#### Claims YTD
Table created via importing WLT claims data export file.

<img src="https://github.com/fdk1034e/Access-Database-Queries/blob/master/298/Claims_YTD.png?raw=true" width="50%"> 

## Step 2 - Pull claims data from MediClaims

MediClaims linked server accessed from:

| Server | Database | Credential |
| --- | --- | --- |
| SQL01_CLT | Master | Windows Auth |

In [1]:
/*Pull claims data from Medi*/
DECLARE @MSTKEY INT = 298
DECLARE @StartDate DATE = '2019-08-01'
DECLARE @EndDate DATE = '2020-01-31'

DELETE
FROM MEDI..JASPER_REPORT.REPORT_PARAMETERS

INSERT INTO MEDI..JASPER_REPORT.REPORT_PARAMETERS (
    GRPNO
    , START_DT
    , END_DT
    )
SELECT @MSTKEY
    , @StartDate
    , @EndDate

IF OBJECT_ID(N'tempdb..#Medi_Claims') IS NOT NULL
    DROP TABLE #Medi_Claims

SELECT *
INTO #Medi_Claims
FROM OPENQUERY(MEDI,
        '
        WITH GrpData as (
            SELECT
                t1.*,
                t2.START_DT,
                t2.END_DT
            FROM MEDI.GROUPDATA t1
                JOIN JASPER_REPORT.REPORT_PARAMETERS t2
                    ON t1.MSTKEY = t2.GRPNO)
        , HstClaim as (
            SELECT DISTINCT
                T.GRPNO Sub_Group_No,
                G.NAME Sub_Group_Name,
                T.LOCNO Location_No,
                L.NAME Location_Name,
                to_Char(E.GRPNO,''FM00000'') || to_char(E.EMPNO,''FM00000'')|| to_char(E.DEPNO,''FM00'') Insured_ID,
                to_Char(E.GRPNO,''FM00000'') || to_char(E.EMPNO,''FM00000'')|| to_char(E.DEPNO,''FM00'') Patient_ID,
                T.CLMPRE ||''-''|| to_char(T.CLMNO,''FM000000000'')||''-''|| to_char(T.CLMSUF,''FM0000'') Claim_Number,
                D.CLMLN Claim_Line_Number,
                DECODE( T.DLYFLAG, 0, ''UNEDITED'', 1, ''ADJUDICATED'', 2, ''THROUGH DAILY'', 3, ''PAID'', 6, ''INSTANT CHECK'', 9, ''DEBIT ADJUSTED'') Claim_Status,
                T.PDDT Paid_Date,
                DECODE( T.CLMTP, 0, ''OTHER'', 1, ''MEDICAL'', 2, ''DENTAL'', 3, ''VISION'', 4, ''DRUG'', 5, ''MISC'', 6, ''MENTAL/NERVOUS'', 7, ''LTD'', 8, ''STD'', 9, ''EXPENSE'', 10, ''FLEX'') Claim_Type,
                DECODE( D.CLMTP, 0, ''OTHER'', 1, ''MEDICAL'', 2, ''DENTAL'', 3, ''VISION'', 4, ''DRUG'', 5, ''MISC'', 6, ''MENTAL/NERVOUS'', 7, ''LTD'', 8, ''STD'', 9, ''EXPENSE'', 10, ''FLEX'') Claim_Line_Type,
                D.PLNRESP Plan_Resp_Amt,
                DECODE( T.BILLTP, 0, ''Professional'', 1, ''Institutional'', 2, ''Dental'') Bill_Type,
                CASE
                    WHEN P1.FED1099ID = 0
                    THEN TO_CHAR(P1.PRVNO, ''FM000000000'')
                    ELSE TO_CHAR(P1.FED1099ID, ''FM000000000'')
                END	AS Provider_TIN,
                D.PLNNO Plan_No,
                PN.NAME Plan_Name,
                D.CVGCD Coverage_No,
                G.MSTKEY,
                G.GRPKEY,
                G.BEGDT Group_Beg_Date
            FROM MEDI.HSTTRAN T
            JOIN GrpData G
                ON (T.GRPNO = G.GRPNO)
            JOIN MEDI.HSTDETL D
                ON (T.CLMPRE = D.CLMPRE
                AND T.CLMNO = D.CLMNO
                AND T.CLMSUF = D.CLMSUF)
            JOIN MEDI.EMPDEP E
                ON (T.GRPNO = E.GRPNO
                AND T.EMPNO = E.EMPNO
                AND T.DEPNO = E.DEPNO)
            LEFT JOIN MEDI.PROVIDER P1
                ON T.PRVNO = P1.PRVNO
                AND T.PRVSUF = P1.IDSUFFIX
            LEFT JOIN MEDI.LOCATION L
                ON T.GRPNO = L.GRPNO
                AND T.LOCNO = L.LOCNO
            LEFT JOIN MEDI.PLANNAME PN
                ON D.PLNNO = PN.PLNNO
            WHERE 1=1
            AND T.PDDT >= TRUNC(G.START_DT)
            AND T.PDDT < TRUNC(G.END_DT)
            AND T.MEMOCLM = ''F'')
        , MstData as (
            SELECT
                t1.MSTKEY Group_No,
                t1.NAME Group_Name,
                t3.*,
                t1.PBEFFDT Plan_Ben_Eff_DT
            FROM MEDI.GROUPDATA t1
                JOIN JASPER_REPORT.REPORT_PARAMETERS t2
                    ON t1.MSTKEY = t2.GRPNO AND t1.GRPKEY = 0
                JOIN HstClaim t3
                    ON t1.MSTKEY = t3.MSTKEY)
        SELECT * FROM MstData
        '
    )


(0 rows affected)

(1 row affected)

(170234 rows affected)

Total execution time: 00:01:36.508

## Create Table: Claims YTD

In [ ]:
IF OBJECT_ID(N'tempdb..#Claims_YTD') IS NOT NULL
    DROP TABLE #Claims_YTD

CREATE TABLE #Claims_YTD
(
	[Group No] INT,
	[Group Name] VARCHAR(50),
	[Sub-group No] INT,
	[Sub-group Name] VARCHAR(50),
	[Location No] INT,
	[Location Name] VARCHAR(50),
	[Insured ID] VARCHAR(25),
	[Patient ID] VARCHAR(25),
	[Claim Number] VARCHAR(70),
	[Claim Line No] INT,
	[Claim Status] VARCHAR(25),
	[Paid Date] DATETIME,
	[Claim Type] VARCHAR(25),
	[Claim Line Type] VARCHAR(25),
	[Plan Resp Amt] MONEY,
	[Bill Type] VARCHAR(25),
	[Provider TIN] VARCHAR(10),
	[Plan No] INT,
	[Plan Name] VARCHAR(50),
	[Coverage No] INT,
	[MSTKEY] INT,
	[GRPKEY] INT,
    [Group Beg Date] DATETIME,
    [Plan Ben Eff DT] DATETIME,
    GRP_EFF_Date DATETIME,
    TIN_Type VARCHAR(15),
    Location_Name2 VARCHAR(50),
    Plan_Name VARCHAR(50),
    Paid_YrMo VARCHAR(20),
    Paid_Start_DT DATE,
    Paid_End_DT DATE
)


## Update Table: Claims YTD


In [ ]:
IF OBJECT_ID(N'tempdb..#298_Locations') IS NOT NULL
BEGIN
    DROP TABLE #298_Locations
END

CREATE TABLE #298_Locations (
    [Location No] NUMERIC
    , [Location Name] VARCHAR(50)
    , [Location Name2] VARCHAR(50)
    )

INSERT INTO #298_Locations (
    [Location No]
    , [Location Name]
    , [Location Name2]
    )
VALUES (
    0
    , 'LOCATION NOT ASSIGNED'
    , 'PHOEBE HEALTH SYSTEM'
    )
    , (
    10
    , 'PHOEBE HOSPITAL - ACTVE'
    , 'PHOEBE HOSPITAL'
    )
    , (
    11
    , 'PHOEBE HOSPITAL - COBRA'
    , 'PHOEBE HOSPITAL'
    )
    , (
    20
    , 'PHOEBE HEALTH SYSTEM - ACTIVE'
    , 'PHOEBE HEALTH SYSTEM'
    )
    , (
    21
    , 'PHOEBE HEALTH SYSTEM - COBRA'
    , 'PHOEBE HEALTH SYSTEM'
    )
    , (
    30
    , 'PHOEBE PHYSICIANS GROUP - ACTIVE'
    , 'PHOEBE PHYSICIANS GROUP'
    )
    , (
    31
    , 'PHOEBE PHYSICIANS GROUP - COBRA'
    , 'PHOEBE PHYSICIANS GROUP'
    )
    , (
    40
    , 'PHOEBE SUMTER - ACTIVE'
    , 'PHOEBE SUMTER'
    )
    , (
    41
    , 'PHOEBE SUMTER - COBRA'
    , 'PHOEBE SUMTER'
    )
    , (
    50
    , 'PHOEBE WORTH - ACTIVE'
    , 'PHOEBE WORTH'
    )
    , (
    51
    , 'PHOEBE WORTH - COBRA'
    , 'PHOEBE WORTH'
    )

SELECT *
FROM #298_Locations
ORDER BY [Location No]


In [ ]:
IF OBJECT_ID(N'tempdb..#298_Sub_groups') IS NOT NULL
BEGIN
    DROP TABLE #298_Sub_groups
END

SELECT DISTINCT Sub_Group_No
    , Sub_Group_Name
    , Plan_Name = CASE
        WHEN LEN(Sub_Group_Name) > LEN('PHOEBE PUTNEY HEALTH SYSTEM')
            THEN LTRIM(RTRIM(SUBSTRING(Sub_Group_Name, LEN('PHOEBE PUTNEY HEALTH SYSTEM') + 1, 7)))
        END
INTO #298_Sub_groups
FROM #Claims_YTD

SELECT * FROM #298_Sub_groups ORDER BY ISNULL(LEFT(Plan_Name, 0), 'a'), Plan_Name, Sub_Group_No